In [125]:
import pandas as pd
import pickle as pkl
import string
import numpy as np; np.random.seed(7)

## Data Loadings

### word 2 vec repre

In [85]:
w2v_map = {}
with open('data/vectors_pruned.200.txt', 'r') as src:
    src = src.read().strip().split('\n')
    for line in src:
        wv = line.strip().split(' ')
        word = wv.pop(0)
        w2v_map[word] = np.array(list(map(float, wv)))

In [99]:
w2i_map = {}
for i, key in enumerate(w2v_map.keys()):
    w2i_map[key] = i

In [101]:
pkl.dump(w2i_map, open('data/word_idx_map.pkl','wb'))

### map Q idx to context

In [96]:
w2v_matrix = np.zeros(( len((w2v_map.keys())), 200 ))
counter = 0
for _, val in w2v_map.items():
    w2v_matrix[counter] = val
    counter += 1

In [102]:
pkl.dump(w2i_map, open('data/w2v_matrix.pkl','wb'))

In [104]:
def w2v(w):
    return w2v_matrix[w2i_map[w]]

In [120]:
translator = str.maketrans('', '', string.punctuation)
context_repre_matrix = {}
with open('data/text_tokenized.txt', 'r') as src:
    src = src.read().strip().split('\n')
    for line in src:
        context = line.strip().split('\t')
        qid = context.pop(0)
        title = context[0].translate(translator).split()
        if len(context) == 1:
            context_repre_matrix[int(qid)] = {'t':title, 'b':None}
        else:
            body = context[1].translate(translator).split()
            context_repre_matrix[int(qid)] = {'t':title, 'b':body}

### pair building --- (q, q+), (q, q-)

In [61]:
train_df = pd.read_csv('data/train_random.txt', header=None, delimiter='\t', names=['Q','Q+','Q-'])

In [35]:
train_df.head()

,Q,Q+,Q-
0,262144,211039,227387 413633 113297 356390 256881 145638 2962...
1,491522,65911,155119 402211 310669 383107 131731 299465 1633...
2,240299,168608 390642,368007 70009 48077 376760 438005 228888 142340...
3,196614,205184,334471 163710 376791 441664 159963 406360 4300...
4,360457,321532,151863 501857 217578 470017 125838 31836 42066...


In [92]:
def build_set_pair_with_idx(df):
    idx_set = {}
    for idx, row in df.iterrows():
        sample_indices = np.random.choice(range(100), size=20)
        random_neg_samples = np.array(row['Q-'].split(' '))[sample_indices]
        idx_set[row['Q']] = {'pos': list(map(int, row['Q+'].split(' '))), 'neg': list(map(int, random_neg_samples))}
    return idx_set

In [93]:
train_idx_set = build_set_pair_with_idx(train_df)

## 